In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
import numpy as np
from sys import stdout

/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


# Preparación del sistema

## Procesando el archivo pdb
Cargamos el pdb del sistema y arreglamos con PDBFixer los posibles defectos como carencia de residuos, terminales o átomos:

In [3]:
system_pdbfixer=m3t.convert('MisL_Phyre.pdb','pdbfixer')

In [4]:
missing_residues     = system_pdbfixer.findMissingResidues()
nonstandard_residues = system_pdbfixer.findNonstandardResidues()
missing_atoms        = system_pdbfixer.findMissingAtoms()

print('Missing residues:')
print(missing_residues)
print('NonStandard residues:')
print(nonstandard_residues)
print('Missing atoms:')
print(missing_atoms)

Missing residues:
None
NonStandard residues:
None
Missing atoms:
None


In [5]:
system_pdbfixer.missingTerminals

{<Residue 504 (PHE) of chain 0>: ['OXT']}

In [6]:
system_pdbfixer.addMissingAtoms()

In [7]:
system_pdbfixer.addMissingHydrogens(pH=7.4)

### Minimizamos el sistema

In [8]:
forcefield = app.ForceField('amber99sbildn.xml')
system = forcefield.createSystem(system_pdbfixer.topology, constraints=app.HBonds)

kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 0*unit.kelvin
pressure    = None

friction   = 1.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = mm.LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

simulation = app.Simulation(system_pdbfixer.topology, system, integrator, platform, properties)

simulation.context.setPositions(system_pdbfixer.positions)
simulation.context.setVelocitiesToTemperature(0*unit.kelvin)

state_pre_min = simulation.context.getState(getEnergy=True)
energy_pre_min = state_pre_min.getPotentialEnergy()
print("Energía potencial antes de la minimización: {}".format(energy_pre_min))

simulation.minimizeEnergy()

state_post_min = simulation.context.getState(getEnergy=True)
energy_post_min = state_post_min.getPotentialEnergy()
print("Energía potencial tras la minimización: {}".format(energy_post_min))

Energía potencial antes de la minimización: 1942911897704305.8 kJ/mol
Energía potencial tras la minimización: -23903.811624422793 kJ/mol


In [18]:
state = simulation.context.getState(getPositions=True)

In [20]:
state.getPositions()

Quantity(value=[(3.3392491596485065, 1.2415460740017572, -1.4757318106238984), (3.361350975541992, 1.217833755308973, -1.3800743831308266), (3.239160325717359, 1.2367268683451886, -1.4883816786228117), (3.3783397940017608, 1.1735157867621862, -1.53933434185371), (3.3890032291077077, 1.3762928977908402, -1.5077457549120714), (3.352047147830014, 1.443111412519856, -1.4299606451537024), (3.330541551738893, 1.4274640515510022, -1.6395573043151153), (3.2492534362345538, 1.5187743403391316, -1.6358903390022075), (3.5426913886919422, 1.3845866174359749, -1.507000316965657), (3.5714849637883503, 1.4895693132516747, -1.5125268509419065), (3.5822589941626717, 1.3347075296978659, -1.5954737053401544), (3.6082969242860905, 1.321888643728279, -1.386365762577448), (3.6458420392899917, 1.190493702250279, -1.3754782297707173), (3.633269598010706, 1.1145013641973263, -1.451183840013961), (3.6441578052235784, 1.3837200520591368, -1.2683009286155524), (3.6372042867740078, 1.481298918283926, -1.2431683630

In [ ]:
state = simulation.context.getState(getPositions=True)

In [ ]:
system_pdbfixer.positions = state.getPositions()

In [ ]:
m3t.view(system_pdbfixer)

In [ ]:
m3t.view(system_pdbfixer)

In [ ]:
from copy import deepcopy as deepcopy

In [ ]:
aa = deepcopy(system_pdbfixer)

# Tiramos de un extremo aplicando restraints y constraints a parte de la molécula.

### Listas de átomos sobre las que aplicar restricciones

Aplicamos restricciones espaciales de tipo harmónico a todos los átomos del dominio barril intra-membrana:

In [ ]:
list_atoms_positions_restrained = m3t.select(system_pdbfixer, 'resid 183 to 504')

In [ ]:
m3t.view(system_pdbfixer, selection=list_atoms_positions_restrained)

Aplicamos restricciones a todas las distancias relativas entre todos los átomos CA y CB correspondientes al "nucleo" del dominio extracelular:

In [ ]:
list_CAs_distances_constrained = m3t.select(system_pdbfixer, 'resid 0 to 180 and name CA')
list_CBs_distances_constrained = m3t.select(system_pdbfixer, 'resid 0 to 180 and name CB')

In [ ]:
view = m3t.view(system_pdbfixer, selection=list_CAs_distances_constrained)
view.clear()
view.add_ball_and_stick()
view

## Creando el sistema a simular

In [ ]:
forcefield = app.ForceField('amber99sbildn.xml')

In [ ]:
system = forcefield.createSystem(system_pdbfixer.topology, constraints=app.HBonds)

### Añadiños las fuerzas externas de posición (restraints) e internas de distancias (constraints)

### Restricciones harmónicas de posición

In [ ]:
harmonic_restraint_potential = "0.5*K*((x-xo)^2 + (y-yo)^2 + (z-zo)^2)"

In [ ]:
harmonic_restraint_force = mm.CustomExternalForce(harmonic_restraint_potential)

In [ ]:
harmonic_restraint_force.addGlobalParameter('K', 500.0 * unit.kilocalories_per_mole/unit.angstrom**2)
harmonic_restraint_force.addPerParticleParameter('xo')
harmonic_restraint_force.addPerParticleParameter('yo')
harmonic_restraint_force.addPerParticleParameter('zo')

In [ ]:
for index in list_atoms_positions_restrained:
    parameters = system_pdbfixer.positions[index].value_in_unit_system(unit.md_unit_system)
    harmonic_restraint_force.addParticle(int(index), parameters)

In [ ]:
system.addForce(harmonic_restraint_force)

### Restricciones harmónicas de distancia

In [ ]:
# https://simtk.org/api_docs/openmm/api4_1/python/classsimtk_1_1openmm_1_1openmm_1_1HarmonicBondForce.html
# https://simtk.org/api_docs/openmm/api4_1/python/classsimtk_1_1openmm_1_1openmm_1_1CustomBondForce.html#_details
harmonic_bond_force = mm.HarmonicBondForce()

In [ ]:
num_atoms_list_CAs = len(list_CAs_distances_constrained)

k_cas = 500.0 * unit.kilocalories_per_mole/unit.angstrom**2

for ii in range(num_atoms_list_CAs):
    for jj in range(ii+1,num_atoms_list_CAs):
        #dist_ii_jj = m3t.distances(system_pdbfixer,ii,jj)
        vect = system_pdbfixer.positions[ii]-system_pdbfixer.positions[jj]
        dist_ii_jj = np.sqrt(vect[0]**2+vect[1]**2+vect[2]**2)
        harmonic_bond_force.addBond(int(ii), int(jj), dist_ii_jj, k_cas)

In [ ]:
system.addForce(harmonic_bond_force)

Otra manera más cómoda de añadir fuerzas es recurrir a la librería `openmmtools`:    
https://openmmtools.readthedocs.io/en/latest/gettingstarted.html#forces    
https://openmmtools.readthedocs.io/en/latest/forces.html    
https://openmmtools.readthedocs.io/en/latest/api/generated/openmmtools.forces.HarmonicRestraintBondForce.html#openmmtools.forces.HarmonicRestraintBondForce    

```python
from openmmtools.forces import HarmonicRestraintBondForce

num_atoms_list_CAs = len(list_CAs_distances_constrained)

k_cas = 500.0 * unit.kilocalories_per_mole/unit.angstrom**2

for ii in range(num_atoms_list_CAs):
    for jj in range(ii+1,num_atoms_list_CAs):
        distance_restraint = HarmonicRestraintBondForce (spring_constant=k_cas,
                                                         restrained_atom_index1=int(ii),
                                                         restrained_atom_index2=int(jj))
        system.addForce(distance_restraint)
```

## Creando la simulación

In [ ]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 0*unit.kelvin
pressure    = None

In [ ]:
friction   = 1.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = mm.LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

In [ ]:
simulation_time = 0.5*unit.nanoseconds
saving_time = 0.001*unit.nanoseconds
simulation_steps = round(simulation_time/step_size)
saving_steps     = round(saving_time/step_size)
num_steps_saved  = round(simulation_steps/saving_steps)

In [ ]:
print('Simulation steps:', simulation_steps)
print('Saving steps:', saving_steps)
print('Saved steps:', num_steps_saved)

In [ ]:
for ii in range(mm.Platform.getNumPlatforms()):
    platform_name  = mm.Platform.getPlatform(ii).getName()
    platform       = mm.Platform.getPlatformByName(platform_name)
    platform_speed = platform.getSpeed()
    print('Plataforma {} con velocidad {}'.format(platform_name,platform_speed))
    del(platform_name, platform, platform_speed)

In [ ]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [ ]:
simulation = app.Simulation(system_pdbfixer.topology, system, integrator, platform, properties)

In [ ]:
simulation.context.setPositions(system_pdbfixer.positions)
simulation.context.setVelocitiesToTemperature(0*unit.kelvin)

In [ ]:
simulation.minimizeEnergy()

In [ ]:
simulation.reporters.append(app.DCDReporter('trajectory.dcd', saving_steps))
simulation.reporters.append(app.StateDataReporter(stdout, 1*saving_steps, step=True,
                            progress=True, remainingTime=True,
                            speed=True, totalSteps=simulation_steps, separator='\t'))

In [ ]:
aa = simulation.context.getState(getPositions=True)

In [ ]:
aa.getPositions()

In [ ]:
simulation.step(simulation_steps)

In [ ]:
for index in restrained_atoms:
        parameters = sampler_state.positions[index,:].value_in_unit_system(unit.md_unit_system)
        restraint_force.addParticle(index, parameters)

Esto podía haber sido hecho de manera más económica haciendo uso de por ejemplo https://openmmtools.readthedocs.io/en/latest/forces.html
o https://openmmtools.readthedocs.io/en/latest/_modules/openmmtools/forcefactories.html

In [ ]:
https://github.com/uibcdf/Academia/blob/master/Din%C3%A1mica_Molecular/Pozo_Harm%C3%B3nico.ipynb
https://github.com/uibcdf/Academia/blob/master/Din%C3%A1mica_Molecular/MetEncefalina.ipynb
https://openmmtools.readthedocs.io/en/latest/_modules/openmmtools/forcefactories.html
http://www.maccallumlab.org/news/2015/1/23/testing
http://mdtraj.org/latest/examples/openmm.html
http://mdtraj.org/development/api/reporters.html
https://openmmtools.readthedocs.io/en/latest/gettingstarted.html